# Решение #5 - Лингвистический анализ

Решение использует следующие алгоритмы:
* Расстояние Левенштейна
* BK-деревья
* Проверка набора слов
* Проверка наборов местоимений
* Проверка отрицания

Решение основано на коррекции всех орфографических ошибок и анализе предложений на смысловое сходство.

In [1]:
from src import *
from tqdm.auto import tqdm
from ipywidgets import FloatProgress

## Загрузка данных

### Загрузка датасета

In [2]:
from src import *
base_data = load_dataset.load('../data/sample.json')

In [3]:
base_data

['Ты нашёл их или нет?',
 'Почему она так со мной поступает?',
 'Никто туда больше не ходит.',
 'У него с собой не было тогда денег.',
 'Почему они с нами так поступают?',
 'Он всю ночь стонал от сильной боли.',
 'Я больше не хочу с тобой играть.',
 'Тому было тогда всего тринадцать лет.',
 'Что сделал Том с деньгами?',
 'Том меня сейчас хочет видеть?',
 'Он даже меня не замечает.',
 'Тебе это всё нравится?',
 'У него тогда не было с собой денег.',
 'Я его больше не увижу.',
 'Почему она так с ней поступает?',
 'Я хотел бы учиться в Бостоне.',
 'Том и Мэри объявили сегодня о своей пбмолвке.',
 'Том этим сейчас занимается.',
 'Я не могу больше ждать.',
 'Мост очень длинный и высокий.',
 'Том был просто не готов.',
 'Пусть свиньи это едят.',
 'Я ничего не хочу делать.',
 'Ты хотел мне рассказать о свободе?',
 'Что пел Джон на сцене?',
 'Я написал влера письмо.',
 'Тому это тоже не нравится.',
 'Сколько сейчас времени в Париже?',
 'Мы не были готовы.',
 'Они их только что нашли.',
 'Тому 

In [4]:
data = [load_dataset.preprocess_text(text) for text in base_data] # Убираем знаки препинания и лишние пробелы

In [5]:
data

['ты нашел их или нет',
 'почему она так со мной поступает',
 'никто туда больше не ходит',
 'у него с собой не было тогда денег',
 'почему они с нами так поступают',
 'он всю ночь стонал от сильной боли',
 'я больше не хочу с тобой играть',
 'тому было тогда всего тринадцать лет',
 'что сделал том с деньгами',
 'том меня сейчас хочет видеть',
 'он даже меня не замечает',
 'тебе это все нравится',
 'у него тогда не было с собой денег',
 'я его больше не увижу',
 'почему она так с ней поступает',
 'я хотел бы учиться в бостоне',
 'том и мэри объявили сегодня о своей пбмолвке',
 'том этим сейчас занимается',
 'я не могу больше ждать',
 'мост очень длинный и высокий',
 'том был просто не готов',
 'пусть свиньи это едят',
 'я ничего не хочу делать',
 'ты хотел мне рассказать о свободе',
 'что пел джон на сцене',
 'я написал влера письмо',
 'тому это тоже не нравится',
 'сколько сейчас времени в париже',
 'мы не были готовы',
 'они их только что нашли',
 'тому было больше не к кому обратить

### Создание орфографического словаря

In [6]:
# Находим уникальные слова в датасете
actual_word_set = set()
for line in data:
    words = line.split()
    for word in words:
        actual_word_set.add(word)

In [7]:
# Загружаем орфографический словарь
vocabulary_source = "../data/russian-utf8.txt"
with open(vocabulary_source, encoding="utf-8") as file:
    vocabulary = [line.rstrip() for line in file]

In [8]:
# Загружаем имена
names_source = "../data/names.txt"
with open(vocabulary_source, encoding="utf-8") as file:
    names = [line.rstrip() for line in file]
vocabulary = vocabulary + names

In [9]:
# Удаляем слова, которые не задействованы в датасета
vocabulary = [item for item in vocabulary if item in actual_word_set]

In [10]:
# Строим BK-дерево
verification_tree = bk_tree.bk_tree(vocabulary)
verification_tree.build()

  0%|          | 0/1058 [00:00<?, ?it/s]

### Исправляем орфографические ошибки датасета

In [11]:
def fix_spelling(line):
    words = line.split()
    for i in range(len(words)):
        words[i] = verification_tree.find_correct(words[i], 2)
    return ' '.join(words)

In [12]:
data = [fix_spelling(line) for line in data]

In [13]:
data

['ты нашел их или нет',
 'почему она так со мной поступает',
 'никто туда больше не ходит',
 'у него с собой не было тогда денег',
 'почему они с нами так поступают',
 'он всю ночь стонал от сильной боли',
 'я больше не хочу с тобой играть',
 'тому было тогда всего тринадцать лет',
 'что сделал том с деньгами',
 'том меня сейчас хочет видеть',
 'он даже меня не замечает',
 'тебе это все нравится',
 'у него тогда не было с собой денег',
 'я его больше не увижу',
 'почему она так с ней поступает',
 'я хотел бы учиться в бостоне',
 'том и мире объявили сегодня о своей помолвке',
 'том этим сейчас занимается',
 'я не могу больше ждать',
 'мост очень длинный и высокий',
 'том был просто не готов',
 'пусть свиньи это едят',
 'я ничего не хочу делать',
 'ты хотел мне рассказать о свободе',
 'что пел джон на сцене',
 'я написал вчера письмо',
 'тому это тоже не нравится',
 'сколько сейчас времени в париже',
 'мы не были готовы',
 'они их только что нашли',
 'тому было больше не к кому обратить

## Функция проверки на схожесть

In [14]:
def build_word_dictionary(line, verification_tree, error_limit):
    words = line.split()
    result = dict()
    for word in words:
        old = result.get(word, 0)
        result[word] = old + 1
    return result

In [15]:
def count_dict_entries(data):
    return sum([value for key, value in data.items()])

In [16]:
def word_set_correspondence(line1, line2, verification_tree, error_limit):
    dict1 = build_word_dictionary(line1, verification_tree, error_limit)
    dict2 = build_word_dictionary(line2, verification_tree, error_limit)
    total_count = count_dict_entries(dict1) + count_dict_entries(dict2)
    intersection_count = 0
    for key, value1 in dict1.items():
        value2 = dict2.get(key, 0)
        intersection_count += min(value1, value2) * 2
    return intersection_count / total_count

def is_rewrite_word_set_correspondence(line1, line2, verification_tree, error_limit, threshold):
    return word_set_correspondence(line1, line2, verification_tree, error_limit) >= threshold

In [17]:
def count_negatives(line):
    words = line.split()
    negations = ["не", "ни"]
    negation_counters = [0] * len(negations)
    for word in words:
        for i in range(len(negations)):
            if word.startswith(negations[i]):
                negation_counters[i] += 1
    return sum(negation_counters)

In [18]:
def check_negative_corresponense(line1, line2):
    return (count_negatives(line1) % 2) == (count_negatives(line2) % 2)

## Группировка предложений

In [19]:
from src import *
from tqdm.auto import tqdm
from ipywidgets import FloatProgress

# Функция группировки строк
def find_groups(data, base_data):
    groups = []                        # Cписок для сохранения групп
    threshold = 0.9                    # Ограничение для алгоритмов верификации рерайтов
    used = [False for _ in range(len(data))]    # Список меток использования строк
    for current_id in tqdm(range(len(data))):
        if (used[current_id]):
            continue
        current_group = []             # Текущая группа
        current_group.append(base_data[current_id])
        used[current_id] = True
        for next_id in range(current_id + 1, len(data)):
            #print("\tNext id:", next_id)
            if (used[next_id]):
                continue
            line1 = data[current_id]
            line2 = data[next_id]
            # Проверка строк с помощью алгоритмов верификации
            if (is_rewrite_word_set_correspondence(line1, line2, verification_tree, 1, threshold) and
               pronouns.check_pronoun_correspondence(line1, line2) and
               check_negative_corresponense(line1, line2)):
                current_group.append(base_data[next_id])  # Добавление строки в текущую группу
                used[next_id] = True
        groups.append(current_group)             # Добавление сформированной группы к результирующему списку
    return groups

In [20]:
groups = find_groups(data, base_data)

  0%|          | 0/412 [00:00<?, ?it/s]

In [21]:
len(groups)

289

In [22]:
groups

[['Ты нашёл их или нет?'],
 ['Почему она так со мной поступает?'],
 ['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Почему они с нами так поступают?'],
 ['Он всю ночь стонал от сильной боли.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Тому было тогда всего тринадцать лет.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Он даже меня не замечает.'],
 ['Тебе это всё нравится?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Почему она так с ней поступает?'],
 ['Я хотел бы учиться в Бостоне.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Том этим сейчас занимается.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высоки

In [23]:
group3 = [group for group in groups if len(group) == 3]

In [24]:
group3

[['Том не хочет никого видеть.',
  'Том не хочет сегодня никого видеть.',
  'Том сегодня не хочет никого видеть.']]

In [84]:
group2 = [group for group in groups if len(group) == 2]

In [85]:
len(group2)

121

In [86]:
group2

[['Никто туда больше не ходит.', 'Никто больше туда не ходит.'],
 ['У него с собой не было тогда денег.',
  'У него тогда не было с собой денег.'],
 ['Я больше не хочу с тобой играть.', 'Я не хочу с тобой больше играть.'],
 ['Что сделал Том с деньгами?', 'Что сделёл Том с деньгами?'],
 ['Том меня сейчас хочет видеть?', 'Том хочет меня сейчас видеть?'],
 ['Я его больше не увижу.', 'Я больше его не увижу.'],
 ['Том и Мэри объявили сегодня о своей пбмолвке.',
  'Том и Мэри объявили сегодня о своей помолвке.'],
 ['Я не могу больше ждать.', 'Я больше не могу ждать.'],
 ['Мост очень длинный и высокий.', 'Мост очень длинный и очень высокий.'],
 ['Пусть свиньи это едят.', 'Пусть это свиньи едят.'],
 ['Ты хотел мне рассказать о свободе?', 'Ты хотел рассказать мне о свободе?'],
 ['Что пел Джон на сцене?', 'Что Джон пел на сцене?'],
 ['Я написал влера письмо.', 'Я написал вчера письмо.'],
 ['Мы не были готовы.', 'Мы были не готовы.'],
 ['Они их только что нашли.', 'Они только что их нашли.'],
 ['

In [26]:
groups2 = solution2.group_by_equal_vote('../data/sample.json')

  0%|          | 0/412 [00:00<?, ?it/s]

In [28]:
groups5 = groups

In [29]:
# Функция поиска разлиций между найденными наборами групп
def find_difference(group1, group2):
    diff1 = [g for g in group1 if g not in group2]
    diff2 = [g for g in group2 if g not in group1]
    return diff1, diff2

In [30]:
groups1 = [set(g) for g in groups2]
groups2 = [set(g) for g in groups5]
diff1, diff2 = find_difference(groups1, groups2)

In [31]:
diff1

[{'Рому было нечего сказать.', 'Тому было нечего сказать.'},
 {'Я рмогу там быть через час.', 'Я смогу там быть через час.'},
 {'Извините, я хотел бы ьто примерить.',
  'Извините, я хотел бы это примерить.'},
 {'Том ничего нам не платит.', 'Тём ничего нам не платит.'},
 {'Какая гора самая высокая в Европе?', 'Лакая гора самая высокая в Европе?'},
 {'Мне придётся это ещм раз обдумать.', 'Мне придётся это ещё раз обдумать.'},
 {'Он не сказал мне ни слова.', 'Он не сказал мно ни слова.'},
 {'Том очень мае помог.', 'Том очень мне помог.'},
 {'Том не живёт в Бостоне.'},
 {'Том уже должен быть дола.', 'Том уже должен быть дома.'},
 {'Том сейчас не живёт в Бостоне.'}]

In [32]:
diff2

[{'Тому было нечего сказать.'},
 {'Я рмогу там быть через час.'},
 {'Извините, я хотел бы ьто примерить.'},
 {'Тём ничего нам не платит.'},
 {'Лакая гора самая высокая в Европе?'},
 {'Извините, я хотел бы это примерить.'},
 {'Том ничего нам не платит.'},
 {'Мне придётся это ещм раз обдумать.'},
 {'Он не сказал мно ни слова.'},
 {'Том очень мае помог.'},
 {'Мне придётся это ещё раз обдумать.'},
 {'Том не живёт в Бостоне.', 'Том сейчас не живёт в Бостоне.'},
 {'Том уже должен быть дома.'},
 {'Я смогу там быть через час.'},
 {'Том очень мне помог.'},
 {'Он не сказал мне ни слова.'},
 {'Какая гора самая высокая в Европе?'},
 {'Том уже должен быть дола.'},
 {'Рому было нечего сказать.'}]

In [34]:
fix_spelling("том уже должен быть дола")

'том уже должен быть дола'

In [35]:
fix_spelling("я рмогу там быть через час")

'я могу там быть через час'

In [37]:
fix_spelling("он не сказал мно ни слова")

'он не сказал мно ни слова'

In [40]:
shorts = [line for line in vocabulary if len(line) <= 3]

In [41]:
shorts

['без',
 'бы',
 'был',
 'в',
 'вам',
 'вас',
 'во',
 'вот',
 'все',
 'всю',
 'вы',
 'где',
 'дал',
 'для',
 'дня',
 'до',
 'его',
 'ей',
 'ем',
 'ему',
 'ест',
 'еще',
 'же',
 'за',
 'и',
 'иди',
 'из',
 'или',
 'им',
 'их',
 'к',
 'как',
 'кем',
 'ко',
 'кто',
 'лет',
 'мае',
 'мне',
 'мно',
 'мог',
 'мои',
 'мой',
 'мы',
 'на',
 'нам',
 'нас',
 'не',
 'ней',
 'нет',
 'ни',
 'ним',
 'но',
 'нос',
 'о',
 'об',
 'он',
 'она',
 'они',
 'оно',
 'от',
 'пел',
 'под',
 'рад',
 'раз',
 'с',
 'сам',
 'со',
 'так',
 'там',
 'тем',
 'том',
 'три',
 'ты',
 'у',
 'уже',
 'час',
 'чем',
 'что',
 'эта',
 'это',
 'юг',
 'я',
 'без',
 'бы',
 'был',
 'в',
 'вам',
 'вас',
 'во',
 'вот',
 'все',
 'всю',
 'вы',
 'где',
 'дал',
 'для',
 'дня',
 'до',
 'его',
 'ей',
 'ем',
 'ему',
 'ест',
 'еще',
 'же',
 'за',
 'и',
 'иди',
 'из',
 'или',
 'им',
 'их',
 'к',
 'как',
 'кем',
 'ко',
 'кто',
 'лет',
 'мае',
 'мне',
 'мно',
 'мог',
 'мои',
 'мой',
 'мы',
 'на',
 'нам',
 'нас',
 'не',
 'ней',
 'нет',
 'ни',
 'н